In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO
import cv2
import os
import json

def detect_people(frame, model, conf_thresh=0.3):
    """
    frame: BGR 이미지 (np.ndarray)
    model: ultralytics.YOLO 인스턴스
    conf_thresh: 신뢰도 임계치

    반환: List[Tuple[x1, y1, x2, y2, conf]]
    """
    results = model(frame)[0]
    people = []
    for box, cls, conf in zip(results.boxes.xyxy, results.boxes.cls, results.boxes.conf):
        if int(cls) == 0 and float(conf) >= conf_thresh:
            x1, y1, x2, y2 = map(int, box)
            people.append((x1, y1, x2, y2, float(conf)))
    return people

if __name__ == "__main__":
    model = YOLO('yolov8n.pt')
    video_path = '/content/output_scenes/Scene_1.mp4'
    cap = cv2.VideoCapture(video_path)

    os.makedirs('frames', exist_ok=True)
    os.makedirs('content', exist_ok=True)

    frame_idx = 0
    detections = {}  # 저장할 JSON 구조

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_idx += 1

        # YOLO로 사람 검출
        people = detect_people(frame, model, conf_thresh=0.5)

        # 10프레임마다 이미지 저장 및 JSON 기록
        if frame_idx % 10 == 0:
            fname = f'frame_{frame_idx:05d}.jpg'
            out_path = os.path.join('frames', fname)
            cv2.imwrite(out_path, frame)

            # JSON에 [x1,y1,x2,y2,conf] 리스트로 저장
            detections[fname] = [
                [x1, y1, x2, y2, conf] for (x1, y1, x2, y2, conf) in people
            ]

    cap.release()

    # content/detections.json에 dump
    json_path = os.path.join('content', 'detections.json')
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(detections, f, ensure_ascii=False, indent=2)

    print(f"저장된 프레임 수: {len(detections)}")
    print(f"바운딩 정보 JSON: {json_path}")


In [ ]:
import zipfile
import os

# Paths
frames_dir = '/content/frames'
json_path = '/content/content/detections.json'
# Changed zip_path to a valid location in Colab
zip_path = '/content/frames_and_detections.zip'

# Create zip archive
with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as z:
    # Add all frames
    for root, dirs, files in os.walk(frames_dir):
        for file in files:
            if file.lower().endswith('.jpg'):
                full_path = os.path.join(root, file)
                arcname = os.path.relpath(full_path, '/content')
                z.write(full_path, arcname=arcname)
    # Add the JSON file
    if os.path.exists(json_path):
        z.write(json_path, arcname=os.path.relpath(json_path, '/content'))
    else:
        print(f"JSON file not found at {json_path}")

print(f"Archive created at: {zip_path}")